In [9]:
!pip3 install tensorflow

     |████████████████████████████████| 239.1 MB 23 kB/s  eta 0:00:011    |████                            | 30.0 MB 1.8 MB/s eta 0:01:59     |█████████▊                      | 72.6 MB 3.6 MB/s eta 0:00:47     |██████████████████▋             | 139.2 MB 1.3 MB/s eta 0:01:19     |███████████████████▊            | 147.0 MB 3.6 MB/s eta 0:00:26     |██████████████████████████▏     | 195.7 MB 564 kB/s eta 0:01:17     |███████████████████████████▍    | 204.2 MB 15.5 MB/s eta 0:00:03     |█████████████████████████████▋  | 221.2 MB 360 kB/s eta 0:00:50
     |████████████████████████████████| 1.7 MB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 24.5 MB 7.0 MB/s eta 0:00:011    |█                               | 829 kB 29.3 MB/s eta 0:00:01     |████                            | 3.1 MB 29.3 MB/s eta 0:00:01███▉                          | 4.5 MB 29.3 MB/s eta 0:00:01     |████████▉                       | 6.7 MB

In [10]:
import pandas as pd
import numpy as np
import json
from tensorflow import keras
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


2024-02-29 12:13:08.246465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
x_src_path = '../Dataset/assignment1New.json'
y_src_path = '../DataBook/Assignment1_Data_Analyst.xlsx'

In [12]:
data = pd.read_json(x_src_path)


In [13]:
df_supervision = pd.read_excel(y_src_path)
plagiarised_array = df_supervision['Plagiarised'].astype(int).values

In [14]:
# X = np.array(X_flattened_padded)
X = data.values
y = plagiarised_array

In [15]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_imputed = imputer.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [17]:
# first, we need to flatten our images (from 2d into a 1d array)
x_train_flat = X_train.reshape(X_train.shape[0], -1)
x_test_flat = X_test.reshape(X_test.shape[0], -1)

# create and train the perceptron
perceptron = Perceptron()
perceptron.fit(x_train_flat, y_train)

# print train and test accuracy scores
print(f"Training data score: {perceptron.score(x_train_flat, y_train)}")
print(f"Training data score: {perceptron.score(x_test_flat, y_test)}")


Training data score: 0.7906976744186046
Training data score: 0.8181818181818182


In [18]:
# let's use tensorflow keras library to build a 10 perceptron model
#  Dense layers are standard fully-interconnected neural network layers
model = keras.models.Sequential([
    keras.layers.Dense(10, activation='softmax') # output layer
])

# we'll use sparse_categorical_crossentropy as the loss function
LOSS_FN = keras.losses.sparse_categorical_crossentropy

# compile the model with standard backprop training algorithm called 'adam'
model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

# train on training data, and validate on test data
# we'll train for 5 epochs
model.fit(x_train_flat, y_train, epochs=5, validation_data=(x_test_flat, y_test))

Epoch 1/5
2/2 [==============================] - 4s 1s/step - loss: 40623.3945 - accuracy: 0.6279 - val_loss: 61139.8359 - val_accuracy: 0.1818
Epoch 2/5
2/2 [==============================] - 0s 373ms/step - loss: 47902.0039 - accuracy: 0.3721 - val_loss: 30782.1992 - val_accuracy: 0.8182
Epoch 3/5
2/2 [==============================] - 0s 118ms/step - loss: 39166.5898 - accuracy: 0.7907 - val_loss: 49240.1289 - val_accuracy: 0.8182
Epoch 4/5
2/2 [==============================] - 0s 148ms/step - loss: 56463.6523 - accuracy: 0.7907 - val_loss: 50480.4531 - val_accuracy: 0.8182
Epoch 5/5
2/2 [==============================] - 0s 117ms/step - loss: 55081.7227 - accuracy: 0.7907 - val_loss: 39489.3711 - val_accuracy: 0.8182


In [19]:
# let's build a whole bunch of models to compare
modelNames = [
    '1d',
    '2d',
    '3d',
    '1c1d',
    '1c2d',
    '2c2d',
]

# in all models we'll have the same number of nodes per hidden layer
NODES_PER_HIDDEN_LAYER = 15

models = [
    # just the output layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 hidden layer
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 2 hidden layers
    keras.models.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),   # hidden layer 2
        keras.layers.Dense(10, activation='softmax') # output layer
    ]),
    # 1 convolutional layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 1 convolutional layer and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ]),
    # 2 convolutional layers and 1 hidden dense layer
    keras.models.Sequential([
        keras.Input(shape=(28,28,1)),
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 1
        keras.layers.Conv2D(NODES_PER_HIDDEN_LAYER, kernel_size=(3, 3), activation="relu"),  #convolutional layer 2
        keras.layers.Flatten(),
        keras.layers.Dense(NODES_PER_HIDDEN_LAYER, activation='relu'),  # hidden layer 1
        keras.layers.Dense(10, activation="softmax")  # output layer
    ])
]



In [20]:
# compile all models
for model in models:
    model.compile(optimizer='adam',loss=LOSS_FN,metrics=['accuracy'])

In [21]:
TRAINING_EPOCHS = 5

# train all models
for model, name in zip(models, modelNames):
    print(f'training model {name}')
    model.fit(X_train, y_train, epochs=TRAINING_EPOCHS)


training model 1d
Epoch 1/5


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(f

In [ ]:
# get all model accuracy scores on test data
scores = [model.evaluate(X_test,y_test)[1] for model in models]


In [ ]:
# let's also add the perceptron score to our accuracy scores list
modelNames.insert(0, 'perceptron')
scores.insert(0, perceptron.score(x_test_flat, y_test) )

# print all models and scores
for name,score in zip(modelNames,scores):
    print(name, score)

In [ ]:
# display as a bar chart
plt.bar(modelNames,scores)
plt.ylim(0.8, 1.0)
plt.show()